In [1]:
# 새로운 가상 환경 media
# python = 3.9
# pip instal mediapipe

import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

In [5]:
# 이미지 잘 로드되는지 확인 
# 구글에서 "손" 검색해서 "손" 사진 "hand.jpg"로 저장
import cv2
hand = cv2.imread("hand.jpg")
cv2.imshow("hand", hand)
cv2.waitKey(0) # 무한 대기 -> 아무키를 누르면 다음 코드로넘어감

cv2.destroyAllWindows() # 창을 다 끔
cv2.waitKey(1)

-1

In [10]:
# cv2 추가
import cv2

# STEP 1: Import the necessary modules.
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

# solutions 추가
from mediapipe import solutions

# pb2 추가
from mediapipe.framework.formats import landmark_pb2

# numpy 추가
import numpy as np

MARGIN = 10  # pixels
FONT_SIZE = 1
FONT_THICKNESS = 1
HANDEDNESS_TEXT_COLOR = (88, 205, 54) # vibrant green

def draw_landmarks_on_image(rgb_image, detection_result):
    hand_landmarks_list = detection_result.hand_landmarks
    handedness_list = detection_result.handedness
    annotated_image = np.copy(rgb_image)

    # Loop through the detected hands to visualize.
    for idx in range(len(hand_landmarks_list)):
        hand_landmarks = hand_landmarks_list[idx]
        handedness = handedness_list[idx]

        # Draw the hand landmarks.
        hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
        hand_landmarks_proto.landmark.extend([
            landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks
        ])
        solutions.drawing_utils.draw_landmarks(
            annotated_image,
            hand_landmarks_proto,
            solutions.hands.HAND_CONNECTIONS,
            solutions.drawing_styles.get_default_hand_landmarks_style(),
            solutions.drawing_styles.get_default_hand_connections_style())

        # Get the top left corner of the detected hand's bounding box.
        height, width, _ = annotated_image.shape
        x_coordinates = [landmark.x for landmark in hand_landmarks]
        y_coordinates = [landmark.y for landmark in hand_landmarks]
        text_x = int(min(x_coordinates) * width)
        text_y = int(min(y_coordinates) * height) - MARGIN

        # Draw handedness (left or right hand) on the image.
        cv2.putText(annotated_image, f"{handedness[0].category_name}",
                    (text_x, text_y), cv2.FONT_HERSHEY_DUPLEX,
                    FONT_SIZE, HANDEDNESS_TEXT_COLOR, FONT_THICKNESS, cv2.LINE_AA)

    return annotated_image

# STEP 2: Create an HandLandmarker object.
base_options = python.BaseOptions(model_asset_path='hand_landmarker.task')
options = vision.HandLandmarkerOptions(base_options=base_options,
                                       num_hands=2)
detector = vision.HandLandmarker.create_from_options(options)

# STEP 3: Load the input image.
image = mp.Image.create_from_file("hand.jpg")

# STEP 4: Detect hand landmarks from the input image.
detection_result = detector.detect(image)

# 이미지를 BGR 형식으로 변환
image_bgr = cv2.cvtColor(image.numpy_view(),cv2.COLOR_RGB2BGR)

# STEP 5: Process the classification result. In this case, visualize it.
annotated_image = draw_landmarks_on_image(image_bgr, detection_result)
cv2.imshow("hand", cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR))
cv2.waitKey(0)

cv2.destroyAllWindows()
cv2.waitKey(1)

-1

In [17]:
# 실시간 웹캠으로 손 랜드마크 찍어보자
import cv2
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np

MARGIN = 10
FONT_SIZE = 1
FONT_THICKNESS = 1
HANDEDNESS_TEXT_COLOR = (88, 205, 54) # vibrant green

def draw_landmarks_on_image(rgb_image, detection_result):
    hand_landmarks_list = detection_result.hand_landmarks
    handedness_list = detection_result.handedness
    annotated_image = np.copy(rgb_image)

    for idx in range(len(hand_landmarks_list)):
        hand_landmarks = hand_landmarks_list[idx]
        handedness = handedness_list[idx]

        hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
        hand_landmarks_proto.landmark.extend([
            landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks
        ])
        solutions.drawing_utils.draw_landmarks(
            annotated_image,
            hand_landmarks_proto,
            solutions.hands.HAND_CONNECTIONS,
            solutions.drawing_styles.get_default_hand_landmarks_style(),
            solutions.drawing_styles.get_default_hand_connections_style())

        height, width, _ = annotated_image.shape
        x_coordinates = [landmark.x for landmark in hand_landmarks]
        y_coordinates = [landmark.y for landmark in hand_landmarks]
        text_x = int(min(x_coordinates) * width)
        text_y = int(min(y_coordinates) * height) - MARGIN

        cv2.putText(annotated_image, f"{handedness[0].category_name}",
                    (text_x, text_y), cv2.FONT_HERSHEY_DUPLEX,
                    FONT_SIZE, HANDEDNESS_TEXT_COLOR, FONT_THICKNESS, cv2.LINE_AA)

    return annotated_image

base_options = python.BaseOptions(model_asset_path='hand_landmarker.task')
options = vision.HandLandmarkerOptions(base_options=base_options,
                                       num_hands=2)
detector = vision.HandLandmarker.create_from_options(options)

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("웹 캠을 열 수 없습니다")
    exit() # 셀 중지
while True:
    ret, frame = cap.read()
    if not ret: # frame이 들어오지 않으면
        break

    # 1) frame을 BGR로 변환
    frame_bgr = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

    # 2) Mediapipe 객체 생성
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame_bgr)

    # 3) 손 랜드마크 감지
    detection_result = detector.detect(mp_image)

    # 4) 랜드마크 그리기
    annotated_image = draw_landmarks_on_image(frame, detection_result)

    # 5) 결과 이미지에 표시
    cv2.imshow("hand_land", annotated_image)

    # q키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# 종료 시 자원 해제
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

In [23]:
# 가위, 바위, 보에 대한 정보가 담겨 있는 csv
!curl -L https://github.com/kairess/Rock-Paper-Scissors-Machine/raw/refs/heads/main/data/gesture_train.csv -o gesture_train.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

100 16990  100 16990    0     0    98k      0 --:--:-- --:--:-- --:--:--   98k


In [26]:
import cv2 # 웹캠 제어 및 ML 사용 
import mediapipe as mp # 손 인식을 할 것
import numpy as np

max_num_hands = 1 # 손은 최대 1개만 인식
gesture = { # **11가지나 되는 제스처 라벨, 각 라벨의 제스처 데이터는 이미 수집됨 (제스처 데이터 == 손가락 관절의 각도, 각각의 라벨)**
    0:'fist', 1:'one', 2:'two', 3:'three', 4:'four', 5:'five',
    6:'six', 7:'rock', 8:'spiderman', 9:'yeah', 10:'ok',
}
rps_gesture = {0:'rock', 5:'paper', 9:'scissors'} # 우리가 사용할 제스처 라벨만 가져옴 

# MediaPipe hands model
mp_hands = mp.solutions.hands # 웹캠 영상에서 손가락 마디와 포인트를 그릴 수 있게 도와주는 유틸리티1
mp_drawing = mp.solutions.drawing_utils # 웹캠 영상에서 손가락 마디와 포인트를 그릴 수 있게 도와주는 유틸리티2

 # 손가락 detection 모듈을 초기화
hands = mp_hands.Hands(  
    max_num_hands=max_num_hands, # 최대 몇 개의 손을 인식? 
    min_detection_confidence=0.5, # 0.5로 해두는 게 좋다!  
    min_tracking_confidence=0.5)  

# 제스처 인식 모델 
file = np.genfromtxt('gesture_train.csv', delimiter=',') # **각 제스처들의 라벨과 각도가 저장되어 있음, 정확도를 높이고 싶으면 데이터를 추가해보자!** 
angle = file[:,:-1].astype(np.float32) # 각도
label = file[:, -1].astype(np.float32) # 라벨
knn = cv2.ml.KNearest_create() # knn(k-최근접 알고리즘)으로   
knn.train(angle, cv2.ml.ROW_SAMPLE, label) # 학습! 

cap = cv2.VideoCapture(0) 

while cap.isOpened(): # 웹캠에서 한 프레임씩 이미지를 읽어옴
    ret, img = cap.read()
    if not ret:
        continue

    img = cv2.flip(img, 1)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    result = hands.process(img)

    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

    # 각도를 인식하고 제스처를 인식하는 부분 
    if result.multi_hand_landmarks is not None: # 만약 손을 인식하면 
        for res in result.multi_hand_landmarks: 
            joint = np.zeros((21, 3)) # joint == 랜드마크에서 빨간 점, joint는 21개가 있고 x,y,z 좌표니까 21,3
            for j, lm in enumerate(res.landmark):
                joint[j] = [lm.x, lm.y, lm.z] # 각 joint마다 x,y,z 좌표 저장

            # Compute angles between joints joint마다 각도 계산 
            # **공식문서 들어가보면 각 joint 번호의 인덱스가 나옴**
            v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19],:] # Parent joint
            v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],:] # Child joint
            v = v2 - v1 # [20,3]관절벡터 
            # Normalize v
            v = v / np.linalg.norm(v, axis=1)[:, np.newaxis] # 벡터 정규화(크기 1 벡터) = v / 벡터의 크기

            # Get angle using arcos of dot product **내적 후 arcos으로 각도를 구해줌** 
            angle = np.arccos(np.einsum('nt,nt->n',
                v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:], 
                v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:])) # [15,]

            angle = np.degrees(angle) # Convert radian to degree

            # Inference gesture 학습시킨 제스처 모델에 참조를 한다. 
            data = np.array([angle], dtype=np.float32)
            ret, results, neighbours, dist = knn.findNearest(data, 3) # k가 3일 때 값을 구한다! 
            idx = int(results[0][0]) # 인덱스를 저장! 

            # # Draw gesture result
            # if idx in rps_gesture.keys(): # 만약 인덱스가 가위바위보 중에 있다면 가위바위보 글씨 표시
            #     cv2.putText(img, text=rps_gesture[idx].upper(), org=(int(res.landmark[0].x * img.shape[1]), int(res.landmark[0].y * img.shape[0] + 20)), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 255, 255), thickness=2)

            # Other gestures 모든 제스처를 표시한다면 
            cv2.putText(img, text=gesture[idx].upper(), org=(int(res.landmark[0].x * img.shape[1]), int(res.landmark[0].y * img.shape[0] + 20)), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 255, 255), thickness=2)

            mp_drawing.draw_landmarks(img, res, mp_hands.HAND_CONNECTIONS) # 손에 랜드마크를 그려줌 

    cv2.imshow('Game', img)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

In [ ]:
import csv
with open("gesture_train.csv","r",encoding="UTF-8") as file:
    reader = csv.reader(file)
    for row in reader:
        print(row)

In [ ]:
# 우리가 원하는 제스처를 랜드마크 좌표와 클래스를 저장하는 로직

# 실시간 웹캠으로 손 랜드마크 찍어보자
import cv2
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np

MARGIN = 10
FONT_SIZE = 1
FONT_THICKNESS = 1
HANDEDNESS_TEXT_COLOR = (88, 205, 54) # vibrant green

def draw_landmarks_on_image(rgb_image, detection_result):
    hand_landmarks_list = detection_result.hand_landmarks
    handedness_list = detection_result.handedness
    annotated_image = np.copy(rgb_image)

    for idx in range(len(hand_landmarks_list)):
        hand_landmarks = hand_landmarks_list[idx]
        handedness = handedness_list[idx]

        hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
        hand_landmarks_proto.landmark.extend([
            landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks
        ])
        solutions.drawing_utils.draw_landmarks(
            annotated_image,
            hand_landmarks_proto,
            solutions.hands.HAND_CONNECTIONS,
            solutions.drawing_styles.get_default_hand_landmarks_style(),
            solutions.drawing_styles.get_default_hand_connections_style())

        height, width, _ = annotated_image.shape
        x_coordinates = [landmark.x for landmark in hand_landmarks]
        y_coordinates = [landmark.y for landmark in hand_landmarks]
        text_x = int(min(x_coordinates) * width)
        text_y = int(min(y_coordinates) * height) - MARGIN

        cv2.putText(annotated_image, f"{handedness[0].category_name}",
                    (text_x, text_y), cv2.FONT_HERSHEY_DUPLEX,
                    FONT_SIZE, HANDEDNESS_TEXT_COLOR, FONT_THICKNESS, cv2.LINE_AA)

    return annotated_image

base_options = python.BaseOptions(model_asset_path='hand_landmarker.task')
options = vision.HandLandmarkerOptions(base_options=base_options,
                                       num_hands=2)
detector = vision.HandLandmarker.create_from_options(options)

# 특정 손 포즈를 취하고
# 클래스화 시켜서 랜드마크 좌표랑 같이 저장!!
import os
csv_file = "data.csv"
file_exists = os.path.isfile(csv_file)

# CSV 파일 헤더 생성
header = []
for i in range(21):
    header.extend([f"x{i}", f"y{i}", f"z{i}"])
header.append("class")

# a모드는 w모드인데, 파일이 이미 존재하면 덮어쓰기
import csv
f = open(csv_file, "a", newline="")
writer = csv.writer(f)

# 만약, 헤더가 없다면 헤더를 추가!!
# 헤더가 없는 지 어떻게 아냐면... 파일이 없으면 헤더가 없는것!
if not file_exists:
    writer.writerow(header)

print("1번키를 누르면 클래스 1로 저장됩니다")
print("2번키를 누르면 클래스 2로 저장됩니다")
print("3번키를 누르면 클래스 3로 저장됩니다")

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("웹 캠을 열 수 없습니다")
    exit() # 셀 중지
while True:
    ret, frame = cap.read()
    if not ret: # frame이 들어오지 않으면
        break

    # 사용자의 key 입력 감지
    key = cv2.waitKey(1) & 0xFF

    # 1) frame을 BGR로 변환
    frame_bgr = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

    # 2) Mediapipe 객체 생성
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame_bgr)

    # 3) 손 랜드마크 감지
    detection_result = detector.detect(mp_image)

    # 해당 랜드마크를 좌표를 찍어서 csv에 행 추가!!
    if detection_result.hand_landmarks: # 손이 감지 되었다면
        hand_land = detection_result.hand_landmarks[0]
        data = []
        for land in hand_land:
            data.extend([land.x, land.y, land.z])

        if key == ord("1"): # 1번 키를 눌렀다면
            data.append(1)
            writer.writerow(data)
            print("클래스 1 저장 완료!!")
        elif key == ord("2"): # 2번 키를 눌렀다면
            data.append(2)s
            writer.writerow(data)
            print("클래스 2 저장 완료!!")
        elif key == ord("3"): # 3번 키를 눌렀다면
            data.append(3)
            writer.writerow(data)
            print("클래스 3 저장 완료!!")

    # 4) 랜드마크 그리기
    annotated_image = draw_landmarks_on_image(frame, detection_result)

    # 5) 결과 이미지에 표시
    cv2.imshow("hand_land", annotated_image)

    # q키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# 종료 시 자원 해제
f.close()
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

In [ ]:
# 1) 우리 랜드마크 21개의 점 가지고 벡터 연산 해서 gesture 추론 로직
# 2) 기존 제스터 모델 사용하려면
# -> data.csv(21 x 3) +1 로 되어있는 것을 -> 각 랜드마크 포인트를 각도로 연산(전처리 과정 진행 필요)